In [1]:
from nba_api.stats.endpoints import playerdashptpass, commonplayerinfo, commonteamroster, playercareerstats
from nba_api.stats.static import teams
import pandas as pd
import time

# "Boston_Celtics", "New_York_Knicks", "Milwaukee_Bucks", "Cleveland_Cavaliers", "Orlando_Magic", "Indiana_Pacers", "Philadelphia_76ers", "Miami_Heat", "Chicago_Bulls", "Atlanta_Hawks", "Brooklyn_Nets", "Toronto_Raptors", "Charlotte_Hornets", "Washington_Wizards", "Detroit_Pistons"

# Fixed season (hardcoded)
SEASON = '2024-25'

# Helper: Clean "Last, First" → "First Last"
def fix_name(name):
    parts = name.split(', ')
    return f"{parts[1]} {parts[0]}" if len(parts) == 2 else name

# # User input
# # inputTeam = input("Input team> ")
# inputTeam = 'Golden State Warriors'
# # Get team info
# nba_teams = teams.get_teams()
# team_info = [team for team in nba_teams if team['full_name'] == inputTeam][0]
# team_id = team_info['id']

# # Get roster
# roster_df = commonteamroster.CommonTeamRoster(team_id=team_id, season=SEASON).get_data_frames()[0]
# players = roster_df[['PLAYER', 'PLAYER_ID']].to_dict(orient='records')
nba_teams = teams.get_teams()

# Create a dictionary mapping team names to their abbreviations
team_name_to_abbreviation = {team['full_name']: team['abbreviation'] for team in nba_teams}

# Print the dictionary
print(team_name_to_abbreviation)


{'Atlanta Hawks': 'ATL', 'Boston Celtics': 'BOS', 'Cleveland Cavaliers': 'CLE', 'New Orleans Pelicans': 'NOP', 'Chicago Bulls': 'CHI', 'Dallas Mavericks': 'DAL', 'Denver Nuggets': 'DEN', 'Golden State Warriors': 'GSW', 'Houston Rockets': 'HOU', 'Los Angeles Clippers': 'LAC', 'Los Angeles Lakers': 'LAL', 'Miami Heat': 'MIA', 'Milwaukee Bucks': 'MIL', 'Minnesota Timberwolves': 'MIN', 'Brooklyn Nets': 'BKN', 'New York Knicks': 'NYK', 'Orlando Magic': 'ORL', 'Indiana Pacers': 'IND', 'Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHX', 'Portland Trail Blazers': 'POR', 'Sacramento Kings': 'SAC', 'San Antonio Spurs': 'SAS', 'Oklahoma City Thunder': 'OKC', 'Toronto Raptors': 'TOR', 'Utah Jazz': 'UTA', 'Memphis Grizzlies': 'MEM', 'Washington Wizards': 'WAS', 'Detroit Pistons': 'DET', 'Charlotte Hornets': 'CHA'}


In [ ]:

# Step 1: Get total minutes for all players
player_minutes = []
print("⏳ Gathering total minutes played by each player...")

for player in players:
    try:
        time.sleep(0.5)
        career = playercareerstats.PlayerCareerStats(player_id=player['PLAYER_ID'])
        df = career.get_data_frames()[0]
        this_season = df[df['SEASON_ID'] == SEASON]

        if not this_season.empty:
            total_minutes = float(this_season['MIN'].values[0])
            player_minutes.append({
                'PLAYER': player['PLAYER'],
                'PLAYER_ID': player['PLAYER_ID'],
                'MIN': total_minutes
            })
            print(f"🟢 {player['PLAYER']} — {total_minutes:.1f} total minutes")
        else:
            print(f"⚠️ No data for {player['PLAYER']} in {SEASON}")

    except Exception as e:
        print(f"❌ Error checking minutes for {player['PLAYER']}: {e}")

# Step 2: Filter top 10 players
sorted_players = sorted(player_minutes, key=lambda x: x['MIN'], reverse=True)
filtered_players = sorted_players[:10]

print("\n✅ Top 10 players by total minutes:")
for p in filtered_players:
    print(f"{p['PLAYER']} — {p['MIN']:.1f} minutes")





⏳ Gathering total minutes played by each player...
🟢 Gary Payton II — 927.0 total minutes
🟢 Jonathan Kuminga — 1144.0 total minutes
⚠️ No data for Taran Armstrong in 2024-25
🟢 Brandin Podziemski — 1716.0 total minutes
🟢 Moses Moody — 1649.0 total minutes
🟢 Kevon Looney — 1142.0 total minutes
🟢 Buddy Hield — 1863.0 total minutes
🟢 Jimmy Butler III — 766.0 total minutes
🟢 Braxton Key — 11.0 total minutes
🟢 Gui Santos — 762.0 total minutes
🟢 Quinten Post — 686.0 total minutes
🟢 Draymond Green — 1983.0 total minutes
🟢 Stephen Curry — 2252.0 total minutes
🟢 Kevin Knox II — 84.0 total minutes
🟢 Trayce Jackson-Davis — 967.0 total minutes
❌ Error checking minutes for Jackson Rowe: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
❌ Error checking minutes for Pat Spencer: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

✅ Top 10 players by total minutes:
Stephen Curry — 2252.0 minutes
Draymond Green — 1983.0 minutes
Bud

In [11]:
# Step 3: Get passing data
all_passes = []

for player in filtered_players:
    try:
        print(f"📦 Processing {player['PLAYER']}...")
        time.sleep(1)

        res = playerdashptpass.PlayerDashPtPass(
            player_id=player['PLAYER_ID'],
            team_id=team_id,
            season=SEASON,
            season_type_all_star='Regular Season'
        )
        df = res.get_data_frames()[0]
        if not df.empty:
            df['Passer'] = player['PLAYER']
            all_passes.append(df[['Passer', 'PASS_TO', 'TEAM_ABBREVIATION', 'PASS', 'AST']])

    except Exception as e:
        print(f"❌ Error for {player['PLAYER']}: {e}")



📦 Processing Stephen Curry...
📦 Processing Draymond Green...
📦 Processing Buddy Hield...
📦 Processing Brandin Podziemski...
📦 Processing Moses Moody...
📦 Processing Jonathan Kuminga...
📦 Processing Kevon Looney...
📦 Processing Trayce Jackson-Davis...
📦 Processing Gary Payton II...
📦 Processing Jimmy Butler III...


In [12]:
# Step 4: Save to CSV
if all_passes:
    pass_df = pd.concat(all_passes)
    pass_df.columns = ['Passer', 'Receiver', 'Team', 'Passes', 'Assists']
    pass_df['Receiver'] = pass_df['Receiver'].apply(fix_name)

    # Filter: keep only rows where both Passer and Receiver are in top 10
    valid_names = {p['PLAYER'] for p in filtered_players}
    pass_df = pass_df[
        pass_df['Passer'].isin(valid_names) & pass_df['Receiver'].isin(valid_names)
    ]
    
    existing_passers = set(pass_df['Passer'].unique())
    missing_passers = valid_names - existing_passers
    print("⚠️ Players in top 10 who had no passing data:", missing_passers)

    # Save file
    pass_df = pass_df.sort_values(by='Passes', ascending=False).reset_index(drop=True)
    csvName = inputTeam.replace(" ", "_") + "_" + SEASON + ".csv"
    filename = 'data\\nba_api\\passing_data\\2024-25\\'+csvName
    pass_df.to_csv(filename, index=False)
    print("\n✅ CSV saved as " + csvName)

    # Print assist totals
    print("\n🧮 Total assists by each passer:")
    assist_totals = pass_df.groupby("Passer")["Assists"].sum().sort_values(ascending=False)
    print(assist_totals)

else:
    print("❌ No passing data found.")

⚠️ Players in top 10 who had no passing data: set()

✅ CSV saved as Golden_State_Warriors_2024-25.csv

🧮 Total assists by each passer:
Passer
Stephen Curry           318
Draymond Green          307
Brandin Podziemski      167
Jimmy Butler III        150
Buddy Hield              92
Kevon Looney             92
Jonathan Kuminga         75
Gary Payton II           73
Moses Moody              70
Trayce Jackson-Davis     61
Name: Assists, dtype: int64
